# CrossEntropy Loss | 交叉熵损失函数

## 1. 交叉熵

在信息学中 **信息熵(entropy)** 是表示系统的混乱程度和确定性的。**交叉熵（Cross Entropy）** 是一种衡量两个概率分布之间的相似程度的指标。
在机器学习中，交叉熵常常用于评估模型预测结果与真实标签之间的差距。交叉熵越小，说明模型预测结果与真实标签越接近，模型性能越好。

交叉熵的公式如下：
$$
H(p,q) = -\sum_{i=1}^{n} p(x_i) \log q(x_i)
$$

其中:
- $p$ 是真实分布
- $x_i$ 是预测样本
- $q$ 是预测分布
- $n$ 是类别数。

## 2. 二分类交叉熵

对于 二分类问题 ，其交叉熵损失计算为：
$$
CE_{loss}(p,y) =
\begin{cases}
     -\log(p), & \text{if } y = 1 \\
    -\log(1-p), & \text{otherwise}
\end{cases}
$$

或者是：
$$CE_{loss} = -[y \cdot log(p) + (1-y) \cdot log(1-p)]$$

其中:
- $p$ 是预测为正例（y=1）的概率
- $y$ 是真实标签（0 或 1）

$p$与$y$的关系为：

$$
p_t = 
\begin{cases}
     p, & \text{if } y = 1 \\
    1-p, & \text{otherwise}
\end{cases}
$$

因此交叉熵也可表示为：
$$CE_{loss}(p, y) = CE_{loss}(p_t) = - \log (p_t)$$

## 3. 多分类交叉熵

多分类交叉熵公式如下：<br>
$$

CE_{loss} = - \sum_{i}^{C} [y_i \cdot log(p_i) + (1-y_i) \cdot log(1-p_i)]
$$

<!-- 如果看作只计算预测正确的概率的交叉熵，那么公式可以简化为：<br>
$$
CE_{loss} = - \sum_{i}^{C} log(p_i)
$$ -->
- $C$ 是类别数
- $y_i$ 是第 $i$ 类的标签，如果第 $i$ 类是正样本，那么 $y_i=1$，否则 $y_i=0$
- $p_i$ 是第 $i$ 类的预测正确的概率
$ 0 < p_i < 1$

## 4. 代码实现


### 4.1 二分类交叉熵实现

In [9]:
# 定义输出和标签

import torch
import torch.nn as nn
import torch.nn.functional as F

input_tensor = torch.tensor([[0.8, 0.2],  # 样本概率
                             [0.3, 0.7], 
                             [0.7, 0.3]]
                             ).float()

label_tensor = torch.tensor([0, 1, 0]) # C =2
# label_tensor = label_tensor.view(-1) # 规定label_tensor的最后维度为类别维度
print(label_tensor.shape)
label_tensor = F.one_hot(label_tensor, num_classes=2).float()

print(input_tensor)
print(label_tensor)

input_tensor.shape, label_tensor.shape

torch.Size([3])
tensor([[0.8000, 0.2000],
        [0.3000, 0.7000],
        [0.7000, 0.3000]])
tensor([[1., 0.],
        [0., 1.],
        [1., 0.]])


(torch.Size([3, 2]), torch.Size([3, 2]))

In [10]:
# 定义自定义的交叉熵损失函数

class CustomCELoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(CustomCELoss, self).__init__()
        self.reduction = reduction

    def forward(self, input_tensor, label_tensor):
        prob_logs = torch.log_softmax(input_tensor, dim=1)
        loss = -torch.sum(label_tensor * prob_logs, dim=1)

        if self.reduction == 'mean':
            return torch.mean(loss)
        elif self.reduction == 'sum':
            return torch.sum(loss)
        else:
            return loss
    
custom_loss_fn = CustomCELoss(reduction='mean')
custom_loss = custom_loss_fn(input_tensor, label_tensor)
print(custom_loss)

tensor(0.4878)


In [11]:
# 与官方的交叉熵损失函数比较
from torch.nn import CrossEntropyLoss

official_loss_fn = CrossEntropyLoss(reduction='mean')
official_loss = official_loss_fn(input_tensor, label_tensor)
print(official_loss)

tensor(0.4878)


### 4.2 多分类交叉熵实现

In [30]:
# 随机生成多维的输入（预测probs）和标签

import torch

input_tensor = torch.rand(1, 4, 128, 128, 128).float()
label_tensor = torch.randint(0, 4, (1, 128, 128, 128))
label_tensor = F.one_hot(label_tensor, num_classes=4).permute(0, 4, 1, 2, 3).float()

print(input_tensor.shape, label_tensor.shape)




torch.Size([1, 4, 128, 128, 128]) torch.Size([1, 4, 128, 128, 128])


In [31]:
custom_loss_fn(input_tensor, label_tensor)

tensor(1.4174)

In [32]:
official_loss_fn(input_tensor, label_tensor)

tensor(1.4174)